# Neuroscape playground

In [1]:
from src.data import get_data
from src.utils import get_args_and_config
from src.train import train
from src.fmri import plot_brain

100%|██████████| 6/6 [00:02<00:00,  2.02it/s]


## data

In [2]:
args, config = get_args_and_config()
data = get_data(args, config)

100%|██████████| 1/1 [00:08<00:00,  8.77s/it]


In [3]:
params_lst = train(data, config)

wandb: Currently logged in as: syrkis. Use `wandb login --relogin` to force relogin


100%|██████████| 6000/6000 [01:34<00:00, 63.48it/s] 


linear_lh_train_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
linear_lh_val_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
linear_rh_train_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
linear_rh_val_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_cat_loss,█▁▁▁▁▁▁▁▁▂▂▂▂▂▂▁▂▂▃▃▂▃▁▃▃▃▂▃▃▃▃▃▃▃▃▃▃▃▃▃
train_lh_corr,▁▂▅▅▆▅▃▅▄▅▅▇▆▆▇▃▇▆▆▇▆▇▇▆▄▇▆▆█▅▇▇▇▇▅▆▇▅▇▇
train_lh_loss,█▂▂▂▂▂▂▃▂▁▁▂▂▁▂▂▃▁▂▂▁▂▁▂▂▂▂▂▂▂▃▁▂▂▂▂▁▂▃▃
train_loss,█▂▂▂▂▂▂▃▂▁▁▂▂▁▂▂▃▁▂▂▁▂▁▂▂▂▂▂▂▂▃▁▂▂▂▂▁▂▃▃
train_rh_corr,▃▃▄▅▅▆▃▃▅▄▇▂▃▆▅▅▃▃▆█▁▅▃▇▅▂▃▁▆▂▅▃▄█▂▂▄▇▆▄
train_rh_loss,█▃▂▂▂▂▂▃▂▁▂▄▄▃▄▄▅▃▅▄▃▄▃▃▃▄▃▄▅▄▅▃▄▅▄▅▃▅▅▆
val_cat_loss,█▁▁▁▁▁▁▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▃▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▂


100%|██████████| 6000/6000 [01:27<00:00, 68.69it/s] 


linear_lh_train_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
linear_lh_val_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
linear_rh_train_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
linear_rh_val_corr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_cat_loss,█▁▁▂▁▂▁▂▁▂▂▂▃▃▂▂▃▃▂▃▂▄▄▃▂▃▃▄▄▃▃▄▄▃▄▃▄▃▃▅
train_lh_corr,▁▂▅▅▅▅▅▇▅▇▆▅▇▆▇▇█▅▆▆▇█▆▄█▅█▆▇▆▇▆▇█▇█▇▆▇▇
train_lh_loss,█▃▂▂▂▂▂▁▂▂▂▂▁▂▂▂▂▂▁▂▁▂▂▂▁▁▁▃▂▁▂▂▂▁▂▂▂▃▂▂
train_loss,█▃▂▂▂▂▂▁▂▂▂▂▁▂▂▂▂▂▁▂▁▂▂▂▁▁▁▃▂▁▂▂▂▁▂▂▂▃▂▂
train_rh_corr,▅▃▄▄▁▆▁▆▄▄▄▅▄▅▅▆▃▆▆▆▅▄▅▇▄▅▃▄▆▅▅▆▃▅█▃▅▄▆▆
train_rh_loss,█▂▂▂▁▂▂▁▁▁▃▂▂▄▃▃▃▃▂▂▃▄▃▃▂▂▃▄▄▃▄▄▄▂▃▃▅▅▃▄
val_cat_loss,█▂▁▁▂▂▂▂▂▂▃▂▂▃▂▂▂▂▃▃▂▂▃▂▃▃▃▃▃▂▂▃▃▃▂▃▃▄▃▃


 91%|█████████ | 5458/6000 [01:20<00:05, 101.08it/s]